In [1]:
import networkx as nx
import numpy as np
import matplotlib.pyplot as plt

# Exercise 1

In [2]:
#state space
nodes = ['o', 'a', 'b', 'c', 'd']

#transition rates matrix
Lambda = np.array([
            # o    a    b    c    d
            [0.0, 2/5, 1/5, 0.0, 0.0],  # o
            [0.0, 0.0, 3/4, 1/4, 0.0],  # a
            [1/2, 0.0, 0.0, 1/3, 0.0],  # b
            [0.0, 0.0, 1/3, 0.0, 2/3],  # c
            [0.0, 1/3, 0.0, 1/3, 0.0]   # d
        ])

n_simulations = 100000



Compute the probability distribution
$$
\bar{\pi}_i(t) = \mathbb P(X(t) = i), \quad i \in \mathcal X
$$

1st approach: global clock with rate $\omega_*$ and matrix $\bar{P}$, where $\bar P$ is the row-stochastic matrix defined as

$$
\bar P_{ij} = \frac{\Lambda_{ij}}{\omega_{*}}, \; i \neq j \quad \bar P_{ii} = 1 - \sum_{i \neq j} \bar P_{ij}
$$

with $\omega = \Lambda \mathbf{1}$ and $\omega_{*}=\max_i \omega_i$

In [3]:
# omega : The total exit rate for each node (sum of the row)
w = np.sum(Lambda, axis=1)

w_star = np.max(w)
# compute the off-diagonal part of P_bar
P_bar = Lambda/w_star 
# add the diagonal part
P_bar = P_bar + np.diag(np.ones(len(w))-np.sum(P_bar,axis=1))

# compute dominant eigenvector
values,vectors = np.linalg.eig(P_bar.T)
index = np.argmax(values.real)
pi_bar = vectors[:,index].real
pi_bar = pi_bar/np.sum(pi_bar)
print("pi_bar=", pi_bar)

nstates = len(pi_bar)

pi_bar= [0.2173913  0.14906832 0.26086957 0.1863354  0.1863354 ]


### (a) Averagae return time to o

Define the system

In [4]:
#initialize simulation parameters

# number of steps
n_steps = 10000
# pos will keep trace of the visited states
pos = np.zeros(n_steps, dtype=int)

pos[0] = 1 # start from node a

t_next = -np.log(np.random.rand())/w_star

In [5]:
# simulation
def simulate_step(current_node_idx, approach):
    if approach == 1:
        pos[current_node_idx] = np.random.choice(nstates, p=P_bar[pos[current_node_idx-1],:])
        t_next = -np.log(np.random.rand())/w_star
    else:
        pos[current_node_idx] = np.random.choice(nstates, p=P[pos[current_node_idx-1],:])
        t_next = -np.log(np.random.rand())/w[pos[current_node_idx]]

    return pos[current_node_idx], t_next


def get_return_time(start_node_idx, approach=1):
    current = start_node_idx
    total_time = 0
    
    # Force the first step (leave the node)
    current, t = simulate_step(current, approach)
    total_time += t
    
    # Walk until we return
    while current != start_node_idx:
        current, t = simulate_step(current, approach)
        total_time += t

    pos[0] = 1 # reset for next simulation
    return total_time    

In [6]:
times_a = [get_return_time(pos[0], 1) for _ in range(n_simulations)]
avg_hitting_o = np.mean(times_a)

print(f"Avg return time for node 'a': {avg_hitting_o:.2f} time units")

Avg return time for node 'a': 6.31 time units


2nd approach: local clocks with rates $\omega_i= \sum_j \Lambda_{ij}$ and matrix $P$, probability to jump to a neighbor $j$ is $P_{ij} = \frac{\Lambda_{ij}}{\omega_i}$


In [7]:
# contruct the P matrix and clock rates w
w = np.sum(Lambda, axis=1)
D = np.diag(w)
P = np.linalg.inv(D) @ Lambda

# number of steps
n_steps = 10000
# pos will keep trace of the visited states
pos = np.zeros(n_steps, dtype=int)

pos[0] = 1 # start from node a


transition_times = np.zeros(n_steps)
t_next = -np.log(np.random.rand())/w[0]

In [8]:
times_a = [get_return_time(pos[0], 2) for _ in range(n_simulations)]
avg_hitting_o = np.mean(times_a)

print(f"Avg return time for node 'a': {avg_hitting_o:.2f} time units")

Avg return time for node 'a': 6.23 time units


#### (b) Compute $\mathbb{E}_a[T^+_a]$
Done analytically and here to compare

In [9]:
import numpy as np

# 1. SETUP THE SYSTEM
# ===================
# ! Repetition, to remove
nodes = ['o', 'a', 'b', 'c', 'd']
target_node_idx = 1  # Node 'a' is index 1

# Re-defining Lambda (based on previous interpretation)
Lambda = np.array([
            # o    a    b    c    d
            [0.0, 2/5, 1/5, 0.0, 0.0],  # o
            [0.0, 0.0, 3/4, 1/4, 0.0],  # a
            [1/2, 0.0, 0.0, 1/3, 0.0],  # b
            [0.0, 0.0, 1/3, 0.0, 2/3],  # c
            [0.0, 1/3, 0.0, 1/3, 0.0]   # d
        ])

# Compute omega (w) and Probability Matrix (P)
w = np.sum(Lambda, axis=1)
P = np.zeros_like(Lambda)
for i in range(len(w)):
    if w[i] > 0:
        P[i, :] = Lambda[i, :] / w[i]

# 2. SOLVE FOR HITTING TIMES (h_i -> a)
# =====================================
# We need to solve for h_o, h_b, h_c, h_d. 
# h_a is 0, so we exclude it from the system of unknowns.

non_target_indices = [i for i in range(len(nodes)) if i != target_node_idx]
n_unknowns = len(non_target_indices)

# ! Shouldnt this be 1 instead of 1/w_a?
# We are building a system A_sys * h_vec = b_vec
# Rearranging the equation: h_i - sum(P_ik * h_k) = 1/w_i
A_sys = np.zeros((n_unknowns, n_unknowns))
b_vec = np.zeros(n_unknowns)

map_idx = {original: new for new, original in enumerate(non_target_indices)}

for i, node_idx in enumerate(non_target_indices):
    # The equation for node_idx:
    # 1 * h_node_idx - sum(P_ik * h_k) = 1/w_node_idx
    
    # Diagonal element (h_node_idx)
    A_sys[i, i] = 1.0
    
    # Right hand side (holding time)
    b_vec[i] = 1.0 / w[node_idx]
    
    # Subtract neighbors' terms
    for neighbor_idx in range(len(nodes)):
        if neighbor_idx == target_node_idx:
            continue # h_a is 0, so P_ik * 0 = 0. No term to subtract.
        
        weight = P[node_idx, neighbor_idx]
        if weight > 0:
            # Find where this neighbor lives in our reduced system
            col_idx = map_idx[neighbor_idx]
            A_sys[i, col_idx] -= weight

# Solve the linear system
h_values_reduced = np.linalg.solve(A_sys, b_vec)

# Map back to full array for clarity (h_a = 0)
h_full = np.zeros(len(nodes))
for i, original_idx in enumerate(non_target_indices):
    h_full[original_idx] = h_values_reduced[i]

# 3. COMPUTE RETURN TIME FOR 'a'
# ==============================
# ! Shouldnt this be 1 instead of 1/w_a?
# E[T_a+] = 1/w_a + sum(P_aj * h_j)
rate_a = w[target_node_idx]
probs_a = P[target_node_idx]

# Weighted average of neighbor hitting times
avg_neighbor_hitting_time = np.sum(probs_a * h_full)

# ! Shouldnt this be 1 instead of 1/w_a?
theoretical_return_a = (1.0 / rate_a) + avg_neighbor_hitting_time

print("--- Theoretical Results ---")
print(f"Node 'a' Exit Rate (w_a): {rate_a}")
print("Hitting times to 'a' from other nodes:")
for i, val in enumerate(h_full):
    if i != target_node_idx:
        print(f"  h_{nodes[i]} -> a : {val:.4f}")

print("-" * 30)
print(f"Theoretical Return Time E_a[T_a+]: {theoretical_return_a:.4f}")

--- Theoretical Results ---
Node 'a' Exit Rate (w_a): 1.0
Hitting times to 'a' from other nodes:
  h_o -> a : 3.5556
  h_b -> a : 5.6667
  h_c -> a : 5.8333
  h_d -> a : 4.4167
------------------------------
Theoretical Return Time E_a[T_a+]: 6.7083


In [14]:
1 / pi_bar

array([4.6       , 6.70833333, 3.83333333, 5.36666667, 5.36666667])

#### c) Average time to hit node d from node o 

##### TODO
- Clean up code above:
  - Merge hitting and return times functions in one
  - Do not use globally defined pos vector
  - Manage the pos vector internally resetting it at every call, and return it when using the above functions
  - Remove repeated code from point (b) (e.g. Lambda, P, omega, ...)

In [ ]:

# ! Repetition, can be merged with previous function
def get_hitting_time(start_node_idx, target_node_idx, approach=1):
    current = start_node_idx
    total_time = 0
    
    # Walk until we reach the target
    while current != target_node_idx:
        current, t = simulate_step(current, approach)
        total_time += t

    return total_time

In [ ]:
n_steps = 10000

# ! This should be done internally, not here (same for the above)
# pos will keep trace of the visited states
pos = np.zeros(n_steps, dtype=int)

pos[0] = 0 # start from node o

In [12]:
times_o = [get_hitting_time(0, 4, 1) for _ in range(n_simulations)]
avg_hitting_o = np.mean(times_o)

print(f"Avg hitting time for node 'o': {avg_hitting_o:.2f} time units")

Avg hitting time for node 'o': 10.74 time units


#### (d) Compute $\mathbb{E}_o[T_d]$
Done analytically and here to compare

In [15]:
import numpy as np

# 1. SETUP THE SYSTEM
# ===================
# ! Repetition, to remove
nodes = ['o', 'a', 'b', 'c', 'd']

target_node_idx = 4  # Node 'd' is index 4
start_node_idx = 0  # We want to know the time starting from 'o'

# Re-defining Lambda (based on previous interpretation)
Lambda = np.array([
            # o    a    b    c    d
            [0.0, 2/5, 1/5, 0.0, 0.0],  # o
            [0.0, 0.0, 3/4, 1/4, 0.0],  # a
            [1/2, 0.0, 0.0, 1/3, 0.0],  # b
            [0.0, 0.0, 1/3, 0.0, 2/3],  # c
            [0.0, 1/3, 0.0, 1/3, 0.0]   # d
        ])

# Compute omega (w) and Probability Matrix (P)
w = np.sum(Lambda, axis=1)
P = np.zeros_like(Lambda)
for i in range(len(w)):
    if w[i] > 0:
        P[i, :] = Lambda[i, :] / w[i]

# 2. SOLVE FOR HITTING TIMES (h_i -> a)
# =====================================
# We need to solve for h_o, h_b, h_c, h_d. 
# h_a is 0, so we exclude it from the system of unknowns.

non_target_indices = [i for i in range(len(nodes)) if i != target_node_idx]
n_unknowns = len(non_target_indices)

# We are building a system A_sys * h_vec = b_vec
# Rearranging the equation: h_i - sum(P_ik * h_k) = 1/w_i
A_sys = np.zeros((n_unknowns, n_unknowns))
b_vec = np.zeros(n_unknowns)

map_idx = {original: new for new, original in enumerate(non_target_indices)}

for i, node_idx in enumerate(non_target_indices):
    # The equation for node_idx:
    # 1 * h_node_idx - sum(P_ik * h_k) = 1/w_node_idx
    
    # Diagonal element (h_node_idx)
    A_sys[i, i] = 1.0
    
    # Right hand side (holding time)
    b_vec[i] = 1.0 / w[node_idx]
    
    # Subtract neighbors' terms
    for neighbor_idx in range(len(nodes)):
        if neighbor_idx == target_node_idx:
            continue # h_a is 0, so P_ik * 0 = 0. No term to subtract.
        
        weight = P[node_idx, neighbor_idx]
        if weight > 0:
            # Find where this neighbor lives in our reduced system
            col_idx = map_idx[neighbor_idx]
            A_sys[i, col_idx] -= weight

# Solve the linear system
h_values_reduced = np.linalg.solve(A_sys, b_vec)

# Map back to full array for clarity (h_a = 0)
h_full = np.zeros(len(nodes))
for i, original_idx in enumerate(non_target_indices):
    h_full[original_idx] = h_values_reduced[i]

# 3. COMPUTE RETURN TIME FOR 'a'
# ==============================
# E[T_a+] = 1/w_a + sum(P_aj * h_j)

print(f"Target Node: {nodes[target_node_idx]}")
print(f"Start Node:  {nodes[start_node_idx]}")
print("-" * 30)

result = h_full[start_node_idx]
print(f"Expected Hitting Time E[T_o -> d]: {result:.4f}")

# Optional: Print all to see the flow
print("\nAll Hitting Times to 'd':")
for i, val in enumerate(h_full):
    print(f"  h_{nodes[i]} -> d : {val:.4f}")

Target Node: d
Start Node:  o
------------------------------
Expected Hitting Time E[T_o -> d]: 10.7667

All Hitting Times to 'd':
  h_o -> d : 10.7667
  h_a -> d : 9.0000
  h_b -> d : 9.3000
  h_c -> d : 4.1000
  h_d -> d : 0.0000


#### (e) French-DeGroot dynamics

In [ ]:
G = nx.DiGraph()
for i, origin in enumerate(nodes):
    for j, target in enumerate(nodes):
        weight = Lambda[i, j]
        if weight > 0:
            G.add_edge(origin, target, weight=weight, label=f"{weight:.2f}")

# Plotting
plt.figure(figsize=(8, 6))
pos = {
    'o': (0, 1),
    'a': (1, 2),
    'b': (1, 0),
    'd': (2, 2),
    'c': (2, 0)
}

# Draw nodes
nx.draw_networkx_nodes(G, pos, node_size=500, node_color='lightcoral')
nx.draw_networkx_labels(G, pos)

# Draw edges
nx.draw_networkx_edges(G, pos, arrowstyle='-|>', arrowsize=20)
edge_labels = nx.get_edge_attributes(G, 'label')
nx.draw_networkx_edge_labels(G, pos, edge_labels=edge_labels)

plt.title("Graph Reconstruction from Matrix Lambda")
plt.axis('off')
plt.show()

The update rule of the node opinion is $x(t+1) = Px(t)$

In [ ]:
w = np.sum(Lambda, axis=1)
P = np.zeros_like(Lambda)
for i in range(len(w)):
    if w[i] > 0:
        P[i, :] = Lambda[i, :] / w[i]

Theorem: if $G=(V,\Epsilon,W)$ possesses a globally reachable aperiodic component $C_0$, then $$\lim_{t\to + \infty}x_i(t)=\pi 'x(0) \: \forall i$$

In [ ]:
# Condition A: Strong Connectivity
is_strongly_connected = nx.is_strongly_connected(G)
print(f"1. Is Strongly Connected? {is_strongly_connected}")

# Condition B: Aperiodicity
is_aperiodic = nx.is_aperiodic(G)
print(f"2. Is Aperiodic? {is_aperiodic}")

#### (f) 
Assume that the initial state of the dynamics for each node $i \in V$ is given by $x_i(0) = ξ_i$, where ${ξ_i} \; i\in V$ are independent random variables with variance $$ \sigma_a^2 = \sigma_b^2 = \sigma_a^2 = \sigma_c^2 = 2, \; \sigma_o^2 = \sigma_d^2 = 1$$  
Compute the variance of the consensus value, and compare your results with the numerical simulations.

In [ ]:
sigmas_sq = np.array([1.0, 2.0, 2.0, 2.0, 1.0]) 
sigmas = np.sqrt(sigmas_sq)

In [ ]:
# 2. theoretical computation
# Compute pi
vals, vecs = np.linalg.eig(P.T)
idx = np.argmin(np.abs(vals - 1))
pi = vecs[:, idx].real
pi = pi / np.sum(pi)
print(pi)

theoretical_var = np.sum((pi**2) * sigmas_sq)
print(f"Theoretical Variance: {theoretical_var:.5f}")

In [ ]:
# simulation

num_trials = 20000
consensus_values = []

for i in range(num_trials):
    x = np.random.normal(0, sigmas, 5)
    if i==0:
        x0 = x
        print(f"Initial state x({i}): {x}")
    for n in range(500):
        x = P @ x
        
    consensus_values.append(x[0])

# 4. RESULTS
empirical_var = np.var(consensus_values)

print(f"Empirical Variance:{empirical_var:.5f}")

Centrality matters!